In [1]:
from cobra.io import read_sbml_model
from cobra import Reaction, Metabolite

In [2]:
Ecoli_model = read_sbml_model('Models/iML1515.xml.gz')

The beta-carotene pathway reactions are added to the *E. coli* iML1515 model to compare the production with an *E. coli* GSM. Firstly, the required reactions and metabolites are defined.

In [7]:
Reaction_1_beta=Reaction('Step1_beta')  # 2 Geranylgeranyl diphosphate <=> Diphosphate + Prephytoene diphosphate
Prephytoene_PP=Metabolite(id='Prephyteoene_pp_c', compartment='c')
Reaction_1_beta.add_metabolites({Ecoli_model.metabolites.gdp_c: -2, #Find name of GPP
                              Prephytoene_PP: 1, # our newly created metabolites
                              Ecoli_model.metabolites.ppi_c: 1 
                             })
Reaction_2_beta=Reaction('Step2_beta')  # Prephytoene diphosphate <=> Diphosphate + 15-cis-Phytoene
Phytoene=Metabolite(id='Phytoene_c', compartment='c')
Reaction_2_beta.add_metabolites({Prephytoene_PP: -1, #made in reaction 1
                              Phytoene: 1, # our newly created metabolite
                              Ecoli_model.metabolites.ppi_c: 1 #Find navn i tabel
                             })
Reaction_3_beta=Reaction('Step3_beta')  # 15-cis-Phytoene + Acceptor <=> all-trans-Phytofluene + Reduced acceptor
Phytofluene=Metabolite(id='Phytofluene_c', compartment='c')
Reaction_3_beta.add_metabolites({Phytoene: -1, #made in reaction 2
                              Phytofluene: 1 # our newly created metabolite
                             })
Reaction_4_beta=Reaction('Step4_beta')  # all-trans-Phytofluene + Acceptor <=> all-trans-zeta-Carotene + Reduced acceptor
zeta_carotene=Metabolite(id='zeta_carotene_c', compartment='c')
Reaction_4_beta.add_metabolites({Phytofluene: -1, #made in reaction 3
                              zeta_carotene: 1 # our newly created metabolite
                             })
Reaction_5_beta=Reaction('Step5_beta')  # all-trans-zeta-Carotene + Reduced acceptor + Oxygen <=> all-trans-Neurosporene + 2 H2O + Acceptor
neurosporene=Metabolite(id='neurosporene_c', compartment='c')
Reaction_5_beta.add_metabolites({zeta_carotene: -1, #made in reaction 4
                                Ecoli_model.metabolites.o2_c: -1, 
                                neurosporene: 1, # our newly created metabolite
                                Ecoli_model.metabolites.h2o_c: 2 
                             })
Reaction_6_beta=Reaction('Step6_beta')  # all-trans-Neurosporene + Reduced acceptor + Oxygen <=> Lycopene + Acceptor + 2 H2O
Lycopene=Metabolite(id='lycopene_c', compartment='c')
Reaction_6_beta.add_metabolites({neurosporene: -1, #made in reaction 5
                                Ecoli_model.metabolites.o2_c: -1, 
                                Lycopene: 1, # our newly created metabolite
                                Ecoli_model.metabolites.h2o_c: 2  
                             })
Reaction_7_beta=Reaction('Step7_beta')  # Lycopene <=> gamma-Carotene
Gamma_carotene=Metabolite(id='gamma_carotene_c', compartment='c')
Reaction_7_beta.add_metabolites({Lycopene: -1, #made in reaction 6
                              Gamma_carotene: 1
                             })
Reaction_8_beta=Reaction('Step8_beta')  # gamma-Carotene <=> beta-Carotene
Beta_carotene=Metabolite(id='Beta_carotene_c', compartment='c')
Reaction_8_beta.add_metabolites({Gamma_carotene: -1, #made in reaction 7
                              Beta_carotene: 1
                             })

After defining the reactions, they are added to the *E. coli* iML1515 model.

In [15]:
Ecoli_model.add_reactions([Reaction_1_beta])
Ecoli_model.add_reactions([Reaction_2_beta])
Ecoli_model.add_reactions([Reaction_3_beta])
Ecoli_model.add_reactions([Reaction_4_beta])
Ecoli_model.add_reactions([Reaction_5_beta])
Ecoli_model.add_reactions([Reaction_6_beta])
Ecoli_model.add_reactions([Reaction_7_beta])
Ecoli_model.add_reactions([Reaction_8_beta])

In [16]:
Ecoli_model.add_boundary(Ecoli_model.metabolites.Beta_carotene_c, type='demand')

Reaction identifier,DM_Beta_carotene_c
Name,demand
Memory address,0x07f037c9d5c40
Stoichiometry,Beta_carotene_c --> -->
GPR,
Lower bound,0
Upper bound,1000.0


The objective of the model is changed to beta-carotene production to calculate the maximal theoretical production. Biomass production is subsequently set as the model objective to calculate the maximal theoretical growth rate for *E. coli* iML1515.

In [21]:
with Ecoli_model:
    Ecoli_model.objective = Ecoli_model.reactions.Step8_beta
    beta_carotene_Ecoli = Ecoli_model.optimize().objective_value
    print('Max. beta-carotene production [mmol gDW^-1 h^-1]:',beta_carotene_Ecoli)
with Ecoli_model:
    Ecoli_model.objective = Ecoli_model.reactions.BIOMASS_Ec_iML1515_core_75p37M
    Max_Ecoli_growth = Ecoli_model.optimize().objective_value
    print('Max. growth rate [h^-1]:',Max_Ecoli_growth)

Max. beta-carotene production [mmol gDW^-1 h^-1]: 2.651034482758614
Max. growth rate [h^-1]: 0.87699721442697


The maximum theoretical yield in *E. coli*, using the iML1515 model, is 2.65 mmol gDW<sup>-1</sup>h<sup>-1</sup>. The maximal growth rate of *E. coli* iML1515 is 0.87 h<sup>-1</sup>.